# CDS: Numerical Methods - Assignment 5

#### See lecture notes and documentation on Brightspace for Python and Jupyter basics. If you are stuck, try to google or get in touch via Discord!

- Solutions must be submitted via Brightspace as a Jupyter notebook (\*.ipynb) before **Wednesday, March 3, 8:30 CET.**

### Handing-in:

1. Click "Kernel -> Restart & Run All"
2. Check all outputs (In[\*] or Out[\*]) for errors and **resolve them if necessary**
3. Click again "Kernel -> Restart & Run All"
4. Save as assignment_05_TEAM-NUMBER.ipynb by clicking "File -> Save as ..."
5. Download your assignment_\*.ipynb file
6. Upload it **in time (before the deadline)** to Brightspace

## 1. Eigenvalues and Eigenvectors

In the following you will implement your own eigenvalue / eigenvector calculation routines based on the inverse power method and the iterated QR decomposition.

### Task 1.1
We start by implementing the inverse power method, which calculates the eigenvector corresponding to an eigenvalue which is closest to a given parameter $\sigma$. In detail, you should implement a Python function $\text{inversePower(A, sigma, eps)}$, which takes as input the $n \times n$ square matrix $A$, the parameter $\sigma$, as well as some accuracy $\varepsilon$. It should return the eigenvector $\mathbf{v}$ (corresponding to the eigenvalue which is closest to $\sigma$) and the number of needed iteration steps.
	
To do so, implement the following algorithm. Start by setting up the needed input:

\begin{align}
    B &= \left( A - \sigma \mathbf{1} \right)^{-1} \\
    \mathbf{b}^{(0)} &= (1,1,1,...)
\end{align}

where $\mathbf{b}_0$ is a vector with $n$ entries. Then repeat the following and increase $k$ each iteration until the error $e$ is smaller than $\varepsilon$:

\begin{align}
    \mathbf{b}^{(k)} &= B \cdot \mathbf{b}^{(k-1)} \\
    \mathbf{b}^{(k)} &= \frac{\mathbf{b}^{(k)}}{|\mathbf{b}^{(k)}|} \\
    e &= \sqrt{ \sum_{i=0}^n \left(|b_i^{(k-1)}| - |b_i^{(k)}|\right)^2 }
\end{align}

Return the last vector $\mathbf{b}^{(k)}$ and the number of needed iterations $k$. 

Verify your implementation by calculating all the eigenvectors of the matrix below and comparing them to the ones calculated by $\text{numpy.linalg.eig()}$. Then implement a unit test for your function.

\begin{align*}
    A = \begin{pmatrix}
    3 & 2 & 1\\ 
    2 & 3 & 2\\
    1 & 2 & 3
    \end{pmatrix}.
\end{align*}

In [1]:
import numpy as np

def error(a, b):
    assert np.shape(a) == np.shape(b), "Vectors are not the same shape"
    return np.sqrt(np.sum((np.abs(a) - np.abs(b))**2))

def inversePower(A, sigma, eps):
    assert np.shape(A)[0] == np.shape(A)[1], "Matrix dimensions don't match"
    B_inv = A - sigma * np.diagflat([1]*np.shape(A)[0])
    B = np.linalg.inv(B_inv)
    b_k = np.ones(np.shape(A)[0])
    b = np.copy(b_k) + 100
    k = 0
    while error(b, b_k) > eps:
        b = np.copy(b_k)
        b_k = np.inner(B, b)
        b_k = b_k / np.linalg.norm(b_k)
        k +=1
    return b_k, k
        
    
    
    

A = np.array([[3,2,1],[2,3,2], [1,2,3]])

print("Found by InversePower:")
for i in [0.6,2.01,6.3]:
    out_new = inversePower(A, i, 10**-10)[0]
    print(out_new * 1/out_new[0])
print("\nFound by numpy:")
a = np.linalg.eig(A)[1].T
for j in a:
    print(j/j[0])
        



Found by InversePower:
[ 1.         -1.68614066  1.        ]
[ 1.0000000e+00  2.9750704e-13 -1.0000000e+00]
[1.         1.18614066 1.        ]

Found by numpy:
[1.         1.18614066 1.        ]
[ 1.00000000e+00  1.11179921e-16 -1.00000000e+00]
[ 1.         -1.68614066  1.        ]


In [2]:
# load ipytest
import ipytest
ipytest.autoconfig()

In [3]:
%%run_pytest[clean]

def test_inversePower():
    A = np.array([[3,2,1],[2,3,2], [1,2,3]])
    a = np.linalg.eig(A)[1].T
    for i,j in enumerate([6.3,2.01,0.6]):
        out_new = inversePower(A, j, 10**-10)[0]
        assert np.allclose(a[i]/a[i,0], out_new/out_new[0],rtol=1e-9,atol=1e-11, equal_nan=False)
    

.                                                                        [100%]
1 passed in 0.08s


### Task 1.2 

Next you will need to implement the tri-diagonalization scheme following Householder. To this end implement a Python function $\text{tridiagonalize(A)}$ which takes a symmetric matrix $A$ as input and returns a tridiagonal matrix $T$ of the same dimension. Therefore, your algorithm should execute the following steps:
	
First use an assertion to make sure $A$ is symmetric. Then let $k$ run from $0$ to $n-1$ and repeat the following:

\begin{align}
    q &= \sqrt{ \sum_{j=k+1}^n \left(A_{j,k}\right)^2 } \\
    \alpha &= -\operatorname{sgn}(A_{k+1,k}) \cdot q \\
    r &= \sqrt{ \frac{ \alpha^2 - A_{k+1,k} \cdot \alpha }{2} } \\
    \mathbf{v} &= \mathbf{0} \qquad \text{... vector of dimension } n \\
    v_{k+1} &= \frac{A_{k+1,k} - \alpha}{2r} \\
    v_{k+j} &= \frac{A_{k+j,k}}{2r}  \quad \text{for } j=2,3,\dots,n \\
    P &= \mathbf{1} - 2 \mathbf{v}\mathbf{v}^T \\
    A &= P \cdot A \cdot P
\end{align}

At the end return $A$ as $T$.

Apply your routine to the matrix $A$ defined in task 1.1 as well as to a few random (but symmetric) matrices of different dimension $n$.

Hint: Use $\text{np.outer()}$ to calculate the *matrix* $\mathbf{v}\mathbf{v}^T$ needed in the definition of the Housholder transformation matrix $P$. 

In [4]:
def tridiagonalize(A):
    assert np.all(A == A.T), "Matrix is not symmetrical"
    assert np.shape(A)[0] == np.shape(A)[1], "Array dimensions don't match"
    n = np.shape(A)[0]
    
    for k in range(n-2):
        q = np.sqrt(np.sum(A[k+1:, k]**2))
        
        alpha = -1* np.sign(A[k+1, k]) * q
        r = np.sqrt((alpha**2 - A[k+1,k] * alpha)/2)
        v = np.zeros(n)
        v[k+1] = (A[k+1, k] - alpha)/(2*r)
        v[k+2:] = (A[k+2:, k])/(2*r)
        
        P = np.diagflat([1]*n)-2*np.outer(v,v.T)
        A = np.dot(P, np.dot(A,P))
        
    return A
        
    # write your Python code here ...
    
    
    

A = np.array([[3,2,1],[2,3,2], [1,2,3]])
print('Original:')
print(A, '\n')
tri_A = tridiagonalize(A)
print('Tridiagonalized:')
print(tri_A, '\n')

for n in range(4,7):
    A = np.random.randint(1,10, (n,n))
    A = np.triu(A) + np.triu(A).T
    print('Original:')
    print(A, '\n')
    tri_A = tridiagonalize(A)
    print('Tridiagonalized:')
    print(np.around(tri_A, 5), '\n')

Original:
[[3 2 1]
 [2 3 2]
 [1 2 3]] 

Tridiagonalized:
[[ 3.         -2.23606798  0.        ]
 [-2.23606798  4.6        -1.2       ]
 [ 0.         -1.2         1.4       ]] 

Original:
[[ 8  2  5  6]
 [ 2 16  9  4]
 [ 5  9 10  3]
 [ 6  4  3 14]] 

Tridiagonalized:
[[ 8.      -8.06226  0.      -0.     ]
 [-8.06226 19.6     -7.91843  0.     ]
 [ 0.      -7.91843 11.35898 -4.67282]
 [-0.       0.      -4.67282  9.04102]] 

Original:
[[12  7  6  3  7]
 [ 7 18  9  5  9]
 [ 6  9  4  5  6]
 [ 3  5  5 18  2]
 [ 7  9  6  2  4]] 

Tridiagonalized:
[[ 12.      -11.95826   0.        0.       -0.     ]
 [-11.95826  27.97203  -8.68641  -0.       -0.     ]
 [  0.       -8.68641   5.42309  -5.21493  -0.     ]
 [  0.        0.       -5.21493  12.5327    1.74245]
 [ -0.       -0.       -0.        1.74245  -1.92781]] 

Original:
[[16  1  2  4  6  1]
 [ 1 18  2  9  2  5]
 [ 2  2  4  2  5  3]
 [ 4  9  2 16  3  8]
 [ 6  2  5  3 16  7]
 [ 1  5  3  8  7 18]] 

Tridiagonalized:
[[ 16.       -7.61577   0.    

### Task 1.3

Implement the $QR$ decomposition based diagonalization routine for tri-diagonal matrices $T$ in Python as a function $\text{QREig(T, eps)}$. It should take a tri-diagonal matrix $T$ and some accuracy $\varepsilon$ as inputs and should return all eigenvalues in the form of a vector $\mathbf{d}$. By making use of the $QR$ decomposition as implemented in Numpy ($\text{numpy.linalg.qr()}$) the algorithm is very simple and reads:

Repeat the following until the error $e$ is smaller than $\varepsilon$:

\begin{align}
    Q \cdot R &= T^{(k)} \qquad \text{... do this decomposition with the help of Numpy!} \\
    T^{(k+1)} &= R \cdot Q \\
    e &= |\mathbf{d_1}| 
\end{align}

where $T^{(0)} = T$ and $\mathbf{d_1}$ is the first sub-diagonal of $T^{(k+1)}$ at each iteration step $k$. At the end return the main-diagonal of $T^{(k+1)}$ as $\mathbf{d}$. 

Implement a unit test for your function based on the matrix $A$ defined in task 1.1. You will need to tri-diagonalize it first.

In [5]:
def QREig(T, eps):
    assert np.allclose(np.triu(T, 2),0), 'Matrix is not tridiagonal'
    assert np.allclose(np.tril(T, -2),0), 'Matrix is not tridiagonal'
    e = 10
    k = 0
    while e > eps:
        Q,R = np.linalg.qr(T)
        T = np.dot(R,Q)
        e = np.linalg.norm(np.diag(T,-1))
        k +=1
    return np.diag(T), k
    
A = np.array([[3,2,1],[2,3,2], [1,2,3]])
print(A)
A_tri = tridiagonalize(A)
eigenvalues = QREig(A_tri, 10**-8)
print(f'The eigenvalues of the matrix are: {eigenvalues[0]}')



[[3 2 1]
 [2 3 2]
 [1 2 3]]
The eigenvalues of the matrix are: [6.37228132 2.         0.62771868]


In [6]:
%%run_pytest[clean]

def test_QREig():
    A = np.array([[3,2,1],[2,3,2], [1,2,3]])
    A_tri = tridiagonalize(A)
    eigenvalues = QREig(A_tri, 10**-8)[0]
    eig_numpy = np.linalg.eig(A)[0]
    assert np.allclose(eig_numpy, eigenvalues,rtol=1e-9,atol=1e-11, equal_nan=False)

.                                                                        [100%]
1 passed in 0.02s


### Task 1.4

With the help of $\text{QREig(T, eps)}$ you can now calculate all eigenvalues. Then you can calculate all corresponding eigenvectors with the help of $\text{inversePower(A, sigma, eps)}$, by setting $\sigma$ to approximately the eigenvalues you found (you should add some small random noise to $\sigma$ in order to avoid singularity issues in the inversion needed for the inverse power method). 

Apply this combination of functions to calculate all eigenvalues and eigenvectors of the matrix $A$ defined in task 1.1.

In [7]:
def eigen_calculate(A, eps):
    A_tri = tridiagonalize(A)
    eigenvalues = QREig(A_tri, eps)[0]
    eigenvectors = np.zeros(np.shape(A))
    for i,eig in enumerate(eigenvalues):
        eigenvectors[i] = inversePower(A, eig + np.random.normal(0,0.1,1), eps)[0]
        eigenvectors[i] = eigenvectors[i]/eigenvectors[i,0]
    return eigenvalues, eigenvectors

A = np.array([[3,2,1],[2,3,2], [1,2,3]])
results = eigen_calculate(A, 10**-8)
print("Matrix\n", A)
print("\nEigenvalues:\n", results[0])
print("\nEigenvectors:\n", results[1])



Matrix
 [[3 2 1]
 [2 3 2]
 [1 2 3]]

Eigenvalues:
 [6.37228132 2.         0.62771868]

Eigenvectors:
 [[ 1.00000000e+00  1.18614066e+00  1.00000000e+00]
 [ 1.00000000e+00  9.61314162e-11 -1.00000000e+00]
 [ 1.00000000e+00 -1.68614066e+00  1.00000000e+00]]


### [Optional] Task 1.5

Test your eigenvalue / eigenvector algorithm for larger random (but symmetric) matrices.

In [8]:
for n in range(4,7):
    A = np.random.randint(1,10, (n,n))
    A = np.triu(A) + np.triu(A, 1).T
    results = eigen_calculate(A, 10**-8)
    print("\nMatrix\n", A)
    print("\nEigenvalues:\n", results[0])
    print("\nEigenvectors:\n", results[1])
    print("-------------------------------------------------------")


Matrix
 [[5 8 4 2]
 [8 5 8 4]
 [4 8 6 3]
 [2 4 3 6]]

Eigenvalues:
 [20.69989253 -4.3877252   4.27335614  1.41447653]

Eigenvectors:
 [[ 1.          1.24289964  1.09074191  0.69686389]
 [ 1.         -1.62961322  0.81206884  0.20045258]
 [ 1.          0.42863613  0.27975428 -2.63737502]
 [ 1.          0.0667731  -1.16324136  0.2666286 ]]
-------------------------------------------------------

Matrix
 [[8 8 5 7 8]
 [8 2 3 8 5]
 [5 3 4 9 6]
 [7 8 9 7 9]
 [8 5 6 9 1]]

Eigenvalues:
 [32.38329561 -6.90795407 -4.6912747   2.93766946 -1.7217363 ]

Eigenvectors:
 [[  1.           0.76585574   0.78058752   1.09145204   0.83916847]
 [  1.          -1.2892273   -0.58479986   1.73313445  -1.72525969]
 [  1.           8.28380446   8.66437786  -3.58485378 -12.1487029 ]
 [  1.           0.48271263  -0.89223877  -0.57417522  -0.0554514 ]
 [  1.          -1.26478528   1.02790744  -0.94128427   0.23074983]]
-------------------------------------------------------

Matrix
 [[2 2 6 8 6 2]
 [2 6 4 6 6 4]


# Weekly Reflection

- How do you judge the level of the last lecture on a scale from 1 to 5?
- with: 1: easy to follow, 3: OK to follow, 5: too complicated to follow

- How long did it take you to complete this assignment?

- How hard was it to finish this assignment on a scale from 1 to 5?
- with: 1: easy / no help needed, 3: could solve with guidance, 5: could not solve it